In [ ]:
#encoding = utf-8
import requests
from requests.exceptions import ConnectionError
import re
import pandas as pd

In [ ]:
def get_course_info(page_number):
    try:
        base = 'http://c.open.163.com/dwr/call/plaincall/OpenSearchBean.searchCourse.dwr'
        payload = {
            'callCount':'1',
            'scriptSessionId':'${scriptSessionId}190',
            'httpSessionId':'e1359886ba9c4cfdaa2973d432aa6e89',
            'c0-scriptName':'OpenSearchBean',
            'c0-methodName':'searchCourse',
            'c0-id':'0',
            'c0-param0':'string:',
            'c0-param1':'number:'+str(page_number),
            'c0-param2':'number:20',
            'batchId':'1489238835634'
        }
        dwr = requests.post(base,data=payload)
        if dwr.status_code == 200:
            return dwr.text
        else:
            return None
    except ConnectionError:
        print('获取课程页面失败！')
        return None

In [ ]:
def parse_course_info(dwr):
    course_pattern = re.compile(r's1\[19\]=s21;(.*)dwr.engine._remoteHandleCallback',re.S)
    courses = re.search(course_pattern,dwr)
    if courses:
        course_temp = courses.group(1).strip()
        course_list = course_temp.split('\n')
        return course_list
    else:
        print('解析课程页面出错！')
        return None

In [ ]:
def parse_course_list(clist):
    course_table = pd.DataFrame({"bigPicUrl":"","category":"","courseId":"","courseType":"","courseUrl":"","description":"","instructor":"","movieCount":"","picUrl":"","school":"","startTime":"","subject":"","tags":"","title":""},index=[])
    for i in range(len(clist)):
        bigPicUrl = re.search('bigPicUrl=(.*?);',clist[i]).group(1).strip('"')
        category = re.search('category=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        courseId = re.search('courseId=(.*?);',clist[i]).group(1).strip('"')
        courseType = re.search('courseType=(.*?);',clist[i]).group(1).strip('"')
        courseUrl = re.search('courseUrl=(.*?);',clist[i]).group(1).strip('"')
        description = re.search('description=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        instructor = re.search('instructor=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        movieCount = re.search('movieCount=(.*?);',clist[i]).group(1).strip('"')
        picUrl = re.search('picUrl=(.*?);',clist[i]).group(1).strip('"')
        school = re.search('school=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        startTime = re.search('startTime=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        subject = re.search('subject=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        tags = re.search('tags=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        title = re.search('title=(.*?);',clist[i]).group(1).strip('"').encode('utf-8').decode('unicode_escape')
        course_table = course_table.append({'bigPicUrl':bigPicUrl,'category':category,'courseId':courseId,'courseType':courseType,'courseUrl':courseUrl,'description':description,'instructor':instructor,'movieCount':movieCount,'picUrl':picUrl,'school':school,'startTime':startTime,'subject':subject,'tags':tags,'title':title},ignore_index=True)
    return course_table

In [ ]:
def main():
    course_table = pd.DataFrame({"bigPicUrl":"","category":"","courseId":"","courseType":"","courseUrl":"","description":"","instructor":"","movieCount":"","picUrl":"","school":"","startTime":"","subject":"","tags":"","title":""},index=[])
    for i in range(250):
        dwr = get_course_info(i)
        clist = parse_course_info(dwr)
        course_temp = parse_course_list(clist)
        course_table = pd.merge(course_table,course_temp,how='outer')
    course_table.to_excel('课程信息.xlsx')

In [ ]:
if __name__=='__main__':
    main()